In [24]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D,BatchNormalization,AveragePooling2D
from keras import regularizers
import numpy as np
import pandas as pd
from keras.models import model_from_json

In [14]:
import  numpy as np
import pandas as pd

In [15]:
filename=r'C:\Users\VergilCrimson\Desktop\fer2013.csv'

In [16]:
with open(filename,'r') as f:
    content = f.readlines()
 
    lines = np.array(content)
 
    num_of_instances = lines.size
    print("number of instances: ",num_of_instances)

number of instances:  35888


In [17]:
df=pd.read_csv(filename)

In [18]:
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [19]:
num_classes=7

In [20]:
x_train, y_train, x_test, y_test = [], [], [], []
 
for i in range(1,num_of_instances):
    emotion, img, usage = lines[i].split(",")
 
    val = img.split(" ")
    pixels = np.array(val, 'float32')
 
    emotion = keras.utils.to_categorical(emotion, num_classes)
 
    if 'Training' in usage:
        y_train.append(emotion)
        x_train.append(pixels)
    elif 'PublicTest' in usage:
        y_test.append(emotion)
        x_test.append(pixels)

In [21]:
x_train=np.array(x_train)
x_test=np.array(x_test)
y_train=np.array(y_train)
y_test=np.array(y_test)

In [22]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((28709, 2304), (3589, 2304), (28709, 7), (3589, 7))

In [25]:
model = Sequential()
 
#1st convolution layer
model.add(Conv2D(64, (5, 5), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(5,5), strides=(2, 2)))
 
#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))
 
#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))
 
model.add(Flatten())
 
#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
 
model.add(Dense(num_classes, activation='softmax'))


In [26]:
batch_size=32
epochs=25

In [27]:
x_train=np.array(x_train)
y_train=np.array(y_train)

x_train=x_train.reshape(-1,48,48,1)
x_test=x_test.reshape(-1,48,48,1)

In [28]:
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [29]:
batch_size=32
epochs=55
model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

Train on 28709 samples, validate on 3589 samples
Epoch 1/55
28709/28709 [==============================] - 14s 497us/step - loss: 1.7704 - acc: 0.2764 - val_loss: 1.6469 - val_acc: 0.3374
Epoch 2/55
28709/28709 [==============================] - 9s 329us/step - loss: 1.6047 - acc: 0.3639 - val_loss: 1.6791 - val_acc: 0.3469
Epoch 3/55
28709/28709 [==============================] - 9s 330us/step - loss: 1.4928 - acc: 0.4150 - val_loss: 1.4389 - val_acc: 0.4494
Epoch 4/55
28709/28709 [==============================] - 9s 330us/step - loss: 1.4169 - acc: 0.4536 - val_loss: 1.3800 - val_acc: 0.4687
Epoch 5/55
28709/28709 [==============================] - 9s 330us/step - loss: 1.3516 - acc: 0.4779 - val_loss: 1.4104 - val_acc: 0.4648
Epoch 6/55
28709/28709 [==============================] - 9s 327us/step - loss: 1.3060 - acc: 0.5017 - val_loss: 1.3302 - val_acc: 0.4776
Epoch 7/55
28709/28709 [==============================] - 9s 331us/step - loss: 1.2567 - acc: 0.5193 - val_loss: 1.3019 - 

In [30]:
scores = model.evaluate(x_test, y_test, verbose=1)
scores

3589/3589 [==============================] - 0s 104us/step


[2.946270553406051, 0.5107272220757321]

In [31]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)


In [32]:
model.save_weights("model.h5")
print("Saved model to disk")


Saved model to disk
